# Naive Bayes

In [1]:
import csv

In [2]:
def carregar(arquivo):
    data = []
    with open(arquivo) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')
        count = 0
        for row in csv_reader:
            if count != 0:
                data.append(tuple(row))
            count += 1
    return(data)

In [3]:
def treinamento(data):
    total = len(data)
    classes = list(set([x[-1] for x in data]))
    combdata = {}
    pdata = {}
    for row in data:
        classe = (row[-1])
        if classe in combdata:
            combdata[classe] += 1
        else:
            combdata[classe] = 1    
        for d in row[:-1]:
            if d in combdata:
                combdata[d] += 1
            else:
                combdata[d] = 1
            key = (d,classe)
            if key in combdata:
                combdata[key] += 1
            else:
                combdata[key] = 1
    # calculando probabilidades
    probdata = {}
    for c in combdata:
        if type(c) == tuple:
            probdata[c] = combdata[c] / combdata[c[1]]
        else:
            probdata[c] = combdata[c] / total
    probdata['classes'] = classes
    return(probdata)

In [4]:
def verificar(probdata, tupla):
    probClasse = {}
    valClasse = 0
    maxClasse = ''
    for classe in probdata['classes']:
        numerador = 1
        denominador = 1
        for v in tupla:
            key = (v, classe)
            if key in probdata:
                numerador = numerador * probdata[key]
            else:
                numerador = numerador * 0.001
            denominador = denominador * probdata[v]
        numerador = numerador * probdata[classe]
        probClasse[classe] = numerador / denominador
        print(classe,probClasse[classe])
        if probClasse[classe] > valClasse:
            valClasse = probClasse[classe]
            maxClasse = classe
    return(maxClasse)

# dataset tempo

In [5]:
tempo =  carregar('data/tempo.csv')
tempo

[('sunny', 'no'),
 ('overcast', 'yes'),
 ('rainny', 'yes'),
 ('sunny', 'yes'),
 ('sunny', 'yes'),
 ('overcast', 'yes'),
 ('rainny', 'no'),
 ('rainny', 'no'),
 ('sunny', 'yes'),
 ('rainny', 'yes'),
 ('sunny', 'no'),
 ('overcast', 'yes'),
 ('overcast', 'yes'),
 ('rainny', 'no')]

In [6]:
modeloTempo = treinamento(tempo)

In [7]:
verificar(modeloTempo, ['sunny'])

yes 0.6
no 0.4


'yes'

In [8]:
verificar(modeloTempo, ['overcast'])

yes 1.0
no 0.00125


'yes'

In [9]:
verificar(modeloTempo, ['rainny'])

yes 0.39999999999999997
no 0.6


'no'

# dataset tenis

In [10]:
tenis = carregar('data/tenis.csv')
tenis

[('sol', 'quente', 'alta', 'falso', 'não'),
 ('sol', 'quente', 'alta', 'verdadeiro', 'não'),
 ('nublado', 'quente', 'alta', 'falso', 'sim'),
 ('chuva', 'agradável', 'alta', 'falso', 'sim'),
 ('chuva', 'fria', 'normal', 'falso', 'sim'),
 ('chuva', 'fria', 'normal', 'verdadeiro', 'não'),
 ('nublado', 'fria', 'normal', 'verdadeiro', 'sim'),
 ('sol', 'agradável', 'alta', 'falso', 'não'),
 ('sol', 'fria', 'normal', 'falso', 'sim'),
 ('chuva', 'agradável', 'normal', 'falso', 'sim'),
 ('sol', 'agradável', 'normal', 'verdadeiro', 'sim'),
 ('nublado', 'agradável', 'alta', 'verdadeiro', 'sim'),
 ('nublado', 'quente', 'normal', 'falso', 'sim'),
 ('chuva', 'agradável', 'alta', 'verdadeiro', 'não')]

In [11]:
modeloTenis = treinamento(tenis)

In [12]:
tupla = ('sol', 'fria', 'alta', 'verdadeiro')
print(verificar(modeloTenis, tupla))

sim 0.2419753086419753
não 0.9408
não


In [13]:
tupla = ('chuva', 'fria', 'normal', 'verdadeiro')
print(verificar(modeloTenis, tupla))

sim 0.7259259259259259
não 0.15680000000000005
sim
